In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow tensorflow-hub pandas

import os
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

# MoveNet Thunder 모델 로드
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']

# 이미지 전처리 함수
def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize_with_pad(img, 256, 256)
    input_img = tf.expand_dims(img, axis=0)
    input_img = tf.cast(input_img, dtype=tf.int32)
    return input_img

# keypoint 추출 함수
def detect_pose(image_path):
    input_img = load_and_preprocess_image(image_path)
    outputs = movenet(input_img)
    keypoints = outputs['output_0'].numpy()
    return keypoints[0, 0, :, :2]  # shape (17, 2)

In [8]:
fall_root = '/content/drive/MyDrive/Colab Notebooks/FallDetection/frames/Fall'  # 너의 경로에 맞게 수정
records = []

for i in range(1, 31):
    folder_name = f'fall_{i:02d}'
    folder_path = os.path.join(fall_root, folder_name)
    if not os.path.exists(folder_path):
        print(f"[❌ 경고] 없음: {folder_path}")
        continue

    for fname in sorted(os.listdir(folder_path)):
        if fname.endswith('.jpg') or fname.endswith('.png'):
            image_path = os.path.join(folder_path, fname)
            try:
                keypoints = detect_pose(image_path)
                for idx, (y, x) in enumerate(keypoints):
                    records.append({
                        'class': 'Fall',
                        'video_folder': folder_name,
                        'frame': fname,
                        'keypoint_index': idx,
                        'x': float(x),
                        'y': float(y)
                    })
            except Exception as e:
                print(f"[⚠️ 오류] {image_path} 실패: {e}")

# 저장
df = pd.DataFrame(records)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/FallDetection/keypoints_fall.csv', index=False)
print("✅ Fall keypoints 저장 완료")


✅ Fall keypoints 저장 완료


In [10]:
sus_root = '/content/drive/MyDrive/Colab Notebooks/FallDetection/frames/Suspicious'  # 경로 확인 필요
records = []

for i in range(1, 22):
    folder_name = f'sus_{i:02d}'
    folder_path = os.path.join(sus_root, folder_name)
    if not os.path.exists(folder_path):
        print(f"[❌ 경고] 없음: {folder_path}")
        continue

    for fname in sorted(os.listdir(folder_path)):
        if fname.endswith('.jpg') or fname.endswith('.png'):
            image_path = os.path.join(folder_path, fname)
            try:
                keypoints = detect_pose(image_path)
                for idx, (y, x) in enumerate(keypoints):
                    records.append({
                        'class': 'Suspicious',
                        'video_folder': folder_name,
                        'frame': fname,
                        'keypoint_index': idx,
                        'x': float(x),
                        'y': float(y)
                    })
            except Exception as e:
                print(f"[⚠️ 오류] {image_path} 실패: {e}")

# 저장
df = pd.DataFrame(records)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/FallDetection/keypoints_suspicious.csv', index=False)
print("✅ Suspicious keypoints 저장 완료")

[❌ 경고] 없음: /content/drive/MyDrive/Colab Notebooks/FallDetection/frames/Suspicious/sus_04
✅ Suspicious keypoints 저장 완료
